In [ ]:
from openff.toolkit import Molecule, Topology

**Note**: The content below demonstrates usage of `nglview` with `openforcefield`. This can be tricky to get working. Install with:

    conda install -c conda-forge nglview
    
And configure for use with Jupyter with:

    jupyter-nbextension enable nglview --py --sys-prefix
    
To use with Jupyterlab, configure with:

    jupyter labextension install  nglview-js-widgets
    jupyter-labextension install @jupyter-widgets/jupyterlab-manager

# Built-in visualizations in `openff-toolkit`

PR [#560](https://github.com/openforcefield/openforcefield/pull/560) introduced rich representation for the `Molecule` objects. This means you can visualize them in your Jupyter Notebooks.

We have implemented three backends:
- rdkit
- openeye
- nglview (requires conformers)

There are two ways to invoke the visualization:
- implicitly, by _evaluating_ the object in a cell
- explicitly, by calling `Molecule.visualize()`

_Implicit_ visualization will try to use `nglview` if there are conformers and fall back to `rdkit` and `openeye` (in that order) if they are not available.

In [ ]:
m = Molecule.from_smiles("CCCCOCC")
m

The regular `display()` call works on `Molecule` objects too.

In [ ]:
display(m)  # noqa

_Explicit_ visualization works as one would expect:

In [ ]:
m.visualize()

This method can take a `backend` parameter, which defaults to `rdkit`:

In [ ]:
m.visualize(backend="rdkit")

`openeye` can also be used, if available:

In [ ]:
try:
    from openeye import oechem

    assert oechem.OEChemIsLicensed()

    m.visualize(backend="openeye")
except (ImportError, AssertionError):
    print('Visualizing with `backend="openeye"` requires the OpenEye Toolkits')

`nglview`, if installed, can only be used if conformers have been generated:

In [ ]:
try:
    m.visualize(backend="nglview")  # this will fail because we have no conformers yet
except ValueError as excinfo:
    # Catch the exception and print its message
    print(str(excinfo))

But, once you generate them, it works! You can zoom in/out, rotate and translate the molecule. You can even inspect the different conformers (if available) using the trajectory player:

In [ ]:
m.generate_conformers()
m.visualize(backend="nglview")

For example, a benzene ring will not have multiple conformers, so you won't see the trajectory player.

In [ ]:
n = Molecule.from_smiles("c1ccccc1")
n.generate_conformers()
n

Notice that, once conformers are available, the implicit representation will use `nglview` to provide a 3D visualization.

It's also possible with any backend to hide non-polar hydrogens, which may produce a more readable image:

In [ ]:
m.visualize(backend="nglview", show_all_hydrogens=False)

In [ ]:
m.visualize(backend="rdkit", show_all_hydrogens=False)

Topologies can also be explicitly visualized:

In [ ]:
from openff.toolkit.utils.utils import get_data_file_path

topology = Topology.from_pdb(
    get_data_file_path("systems/test_systems/T4_lysozyme_water_ions.pdb")
)

In [ ]:
w = topology.visualize()
w

However, they don't support implicit visualization for performance reasons. Note that, again for performance reasons, `Topology` by default does not guarantee to correctly represent connectivity; to gain this guarantee, set the `ensure_correct_connectivity` argument to `True`:

In [ ]:
w = topology.visualize(ensure_correct_connectivity=True)
w